.. _labels-and-legends:

# Labels and Legends

Of course, most figures must be properly labelled before they can be of value, so Toyplot provides several mechanisms to help:

## Coordinate System Labels

First, :ref:`cartesian-coordinates`, :ref:`numberline-coordinates`, and :ref:`table-coordinates` provide labels that can be specified when they are created.  In all cases the `label` parameter provides a top-level label for the coordinate system:

In [1]:
import numpy
import toyplot

canvas = toyplot.Canvas(width=600, height=600)
canvas.cartesian(grid=(2,2,0), label="Cartesian Coordinates").plot(numpy.linspace(0, 1)**2)
canvas.numberline(grid=(2,2,1), label="Numberline Coordinates").scatterplot(numpy.random.normal(size=100))
canvas.table(grid=(2,2,2), label="Table Coordinates", data = numpy.random.random((4, 3)));

<path d="M 50.0 250.0 L 54.0 249.9167013744273 L 58.0 249.66680549770928 L 62.0 249.25031236984589 L 66.0 248.66722199083716 L 70.0 247.91753436068305 L 74.0 247.0012494793836 L 78.0 245.91836734693877 L 82.0 244.6688879633486 L 86.0 243.25281132861306 L 90.0 241.6701374427322 L 94.0 239.92086630570594 L 98.0 238.00499791753435 L 102.0 235.92253227821743 L 106.0 233.67346938775512 L 110.0 231.25780924614745 L 114.0 228.67555185339444 L 118.0 225.92669720949604 L 122.0 223.01124531445231 L 126.0 219.92919616826322 L 130.0 216.68054977092879 L 134.0 213.26530612244898 L 138.0 209.68346522282383 L 142.0 205.93502707205332 L 146.0 202.01999167013744 L 150.0 197.93835901707621 L 154.0 193.69012911286964 L 158.0 189.27530195751771 L 162.0 184.69387755102045 L 166.0 179.94585589337777 L 170.0 175.03123698458978 L 174.0 169.95002082465641 L 178.0 164.7022074135777 L 182.0 159.2877967513536 L 186.0 153.70678883798416 L 190.0 147.9591836734694 L 194.0 142.04498125780927 L 198.0 135.96418159100375 L 202.0 129.7167846730529 L 206.0 123.30279050395671 L 210.0 116.72219908371514 L 214.0 109.97501041232822 L 218.0 103.06122448979593 L 222.0 95.980841316118301 L 226.0 88.733860891295336 L 230.0 81.320283215326981 L 234.0 73.740108288213264 L 238.0 65.993336109954186 L 242.0 58.079966680549774 L 246.0 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 10 20 30 40 50 0.0 0.5 1.0 Cartesian Coordinates -2 0 2 Numberline Coordinates Table Coordinates 0 1 2 0 . 486711 0 . 804376 0 . 763468 0 . 684023 0 . 956016 0 . 370899 0 . 782233 0 . 0025611 0 . 783379 0 . 731204 0 . 377741 0 . 14223

Naturally, some coordinate systems - such as Cartesian - allow you to specify additional, axis-specific labels:

In [2]:
canvas = toyplot.Canvas(width=300, height=300)
axes = canvas.cartesian(label="Cartesian Coordinates", xlabel="Days", ylabel="Users")
axes.plot(numpy.linspace(0, 1)**2);

<path d="M 50.0 250.0 L 54.0 249.9167013744273 L 58.0 249.66680549770928 L 62.0 249.25031236984589 L 66.0 248.66722199083716 L 70.0 247.91753436068305 L 74.0 247.0012494793836 L 78.0 245.91836734693877 L 82.0 244.6688879633486 L 86.0 243.25281132861306 L 90.0 241.6701374427322 L 94.0 239.92086630570594 L 98.0 238.00499791753435 L 102.0 235.92253227821743 L 106.0 233.67346938775512 L 110.0 231.25780924614745 L 114.0 228.67555185339444 L 118.0 225.92669720949604 L 122.0 223.01124531445231 L 126.0 219.92919616826322 L 130.0 216.68054977092879 L 134.0 213.26530612244898 L 138.0 209.68346522282383 L 142.0 205.93502707205332 L 146.0 202.01999167013744 L 150.0 197.93835901707621 L 154.0 193.69012911286964 L 158.0 189.27530195751771 L 162.0 184.69387755102045 L 166.0 179.94585589337777 L 170.0 175.03123698458978 L 174.0 169.95002082465641 L 178.0 164.7022074135777 L 182.0 159.2877967513536 L 186.0 153.70678883798416 L 190.0 147.9591836734694 L 194.0 142.04498125780927 L 198.0 135.96418159100375 L 202.0 129.7167846730529 L 206.0 123.30279050395671 L 210.0 116.72219908371514 L 214.0 109.97501041232822 L 218.0 103.06122448979593 L 222.0 95.980841316118301 L 226.0 88.733860891295336 L 230.0 81.320283215326981 L 234.0 73.740108288213264 L 238.0 65.993336109954186 L 242.0 58.079966680549774 L 246.0 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 10 20 30 40 50 Days 0.0 0.5 1.0 Users Cartesian Coordinates

## Coordinate System Text

Another option for labelling a figure is to insert text using the same domain as the data.  For example, we can label individual series in a plot:

In [3]:
def series(x):
    return numpy.cumsum(numpy.random.normal(loc=0.05, size=len(x)))

numpy.random.seed(1234)
x = numpy.arange(100)
y = numpy.column_stack([series(x) for i in range(5)])

In [4]:
label_style = {"text-anchor":"start", "-toyplot-anchor-shift":"5px"}
canvas, axes, mark = toyplot.plot(x, y)
for i in range(y.shape[1]):
    axes.text(x[-1], y[-1,i], "Series %s" % i, style=label_style)

<path d="M 50.0 343.33002059537489 L 54.599597325492383 358.7920369979883 L 59.199194650984758 338.69902399829283 L 63.798791976477141 342.25837056841618 L 68.39838930196953 351.3459033202555 L 72.997986627461913 338.645870513082 L 77.597583952954295 326.31951593973241 L 82.197181278446678 334.26783253218389 L 86.79677860393906 333.37754329294927 L 91.396375929431429 363.09187540771052 L 95.995973254923825 346.82950264602482 L 100.59557058041619 332.70949416307025 L 105.19516790590859 319.11287313250273 L 109.79476523140096 345.82647843101597 L 114.39436255689336 349.67617383654868 L 118.99395988238572 348.96988796792573 L 123.59355720787812 342.79777741991239 L 128.19315453337049 338.20254793539988 L 132.79275185886289 319.62119641700792 L 137.39234918435525 339.90662258741213 L 141.99194650984765 341.97522053052802 L 146.59154383534002 350.18705885629316 L 151.19114116083239 346.88831609324103 L 155.79073848632478 338.71076911964946 L 160.39033581181715 320.17016227750582 L 164.98993313730955 325.85240881431139 L 169.58953046280195 316.02000903823063 L 174.18912778829431 339.96600901716192 L 178.78872511378671 341.76983923118337 L 183.38832243927908 326.74156137410824 L 187.98791976477148 331.45534391905619 L 192.58751709026387 326.20495393882919 L 197.18711441575624 311.33103683791296 L 201.78671174124861 296.47934861757261 L 206.38630906674095 284.09704124947649 L 210.98590639223335 285.07399555091905 L 215.58550371772574 282.70636005311985 L 220.18510104321811 286.40315901074212 L 224.78469836871051 274.31956327587511 L 229.38429569420288 241.24070655410159 L 233.98389301969527 239.53050351805496 L 238.58349034518764 246.52915869095634 L 243.18308767068004 245.36179987328694 L 247.78268499617243 272.8034333998005 L 252.3822823216648 268.76788040910606 L 256.9818796471572 280.24818814691611 L 261.58147697264957 281.4243947407773 L 266.18107429814194 280.4989687201691 L 270.7806716236343 269.58434164747058 L 275.38026894912673 265.98953487895074 L 279.9798662746191 253.91496338192485 L 284.57946360011147 272.83038673960164 L 289.17906092560389 291.15175252455737 L 293.77865825109626 291.84177427213211 L 298.37825557658863 298.59374362393095 L 302.97785290208105 299.875986880472 L 307.57745022757342 294.40087550655176 L 312.17704755306579 294.20455419855818 L 316.77664487855822 285.86033106372054 L 321.37624220405058 264.23664291281534 L 325.97583952954295 276.76149985665938 L 330.57543685503532 277.03720496146769 L 335.17503418052775 272.18616357938197 L 339.77463150602011 274.33407127479313 L 344.37422883151248 259.64686559487825 L 348.97382615700485 291.49921162485907 L 353.57342348249722 263.30375797759905 L 358.17302080798959 278.11063159573075 L 362.7726181334819 274.56169953634054 L 367.37221545897438 264.33404725003658 L 371.97181278446669 274.30035184363931 L 376.57141010995906 267.36113649325932 L 381.17100743545149 257.1401569864081 L 385.77060476094385 249.3682213985208 L 390.37020208643622 261.24284645102972 L 394.96979941192859 233.35583570526822 L 399.56939673742102 229.60255736184212 L 404.16899406291338 244.54532372648018 L 408.76859138840575 235.30342889360318 L 413.36818871389818 234.09039110820942 L 417.96778603939055 227.11956552076202 L 422.56738336488291 274.73324606582531 L 427.16698069037528 256.15260704391358 L 431.76657801586771 253.40664414882914 L 436.36617534136008 250.49943120200186 L 440.96577266685244 255.65032603189297 L 445.56536999234487 244.57369323610104 L 450.16496731783724 230.54890055625543 L 454.76456464332961 226.20106985300572 L 459.36416196882197 206.65988744320293 L 463.9637592943144 204.90031974793388 L 468.56335661980677 209.64289061264935 L 473.16295394529914 222.89430468552675 L 477.76255127079156 230.14057802127687 L 482.36214859628393 218.3968664165042 L 486.9617459217763 218.82979924903583 L 491.56134324726867 222.82434266348673 L 496.16094057276109 214.98762814843693 L 500.76053789825346 228.79652991325702 L 505.36013522374583 235.05574858979216" style="stroke:rgb(40%,76.

Note that we are using the last point in each series as the anchor for the corresponding label - by default, Toyplot renders text centered on its anchor, so in this case we've chosen a text style that left-aligns the text and offsets it slightly to avoid overlapping the data.

## Canvas Text

When adding text to axes, you specify the text coordinates using the same domain as your data.  Naturally, this limits the added text to the bounds defined by the axes.  For the ultimate in labeling flexibility, you can add text to the canvas directly, using canvas units, outside and/or overlapping coordinate systems:

In [5]:
label_style={"font-size":"18px", "font-weight":"bold"}

canvas = toyplot.Canvas(width=600, height=300)
canvas.cartesian(grid=(1,2,0)).plot(numpy.linspace(1, 0)**2)
canvas.cartesian(grid=(1,2,1), yshow=False).plot(numpy.linspace(0, 1)**2)
canvas.text(300, 120, "This label overlaps two sets of axes!", style=label_style);

<path d="M 50.0 50.0 L 54.0 58.079966680549774 L 58.0 65.993336109954186 L 62.0 73.740108288213221 L 66.0 81.320283215326938 L 70.0 88.733860891295294 L 74.0 95.980841316118301 L 78.0 103.0612244897959 L 82.0 109.97501041232817 L 86.0 116.72219908371511 L 90.0 123.30279050395669 L 94.0 129.7167846730529 L 98.0 135.96418159100375 L 102.0 142.04498125780924 L 106.0 147.9591836734694 L 110.0 153.70678883798413 L 114.0 159.2877967513536 L 118.0 164.70220741357767 L 122.0 169.95002082465641 L 126.0 175.03123698458975 L 130.0 179.94585589337777 L 134.0 184.69387755102045 L 138.0 189.27530195751768 L 142.0 193.69012911286964 L 146.0 197.93835901707621 L 150.0 202.01999167013744 L 154.0 205.93502707205329 L 158.0 209.68346522282383 L 162.0 213.265306122449 L 166.0 216.68054977092876 L 170.0 219.92919616826322 L 174.0 223.01124531445231 L 178.0 225.92669720949601 L 182.0 228.67555185339441 L 186.0 231.25780924614742 L 190.0 233.67346938775509 L 194.0 235.9225322782174 L 198.0 238.00499791753435 L 202.0 239.92086630570594 L 206.0 241.67013744273217 L 210.0 243.25281132861306 L 214.0 244.6688879633486 L 218.0 245.91836734693877 L 222.0 247.00124947938357 L 226.0 247.91753436068305 L 230.0 248.66722199083716 L 234.0 249.25031236984589 L 238.0 249.66680549770925 L 242.0 249.9167013744273 L 246.0 250.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 10 20 30 40 50 0.0 0.5 1.0 <path d="M 350.0 250.0 L 354.0 249.9167013744273 L 358.0 249.66680549770928 L 362.0 249.25031236984589 L 366.0 248.66722199083716 L 370.0 247.91753436068305 L 374.0 247.0012494793836 L 378.0 245.91836734693877 L 382.0 244.6688879633486 L 386.0 243.25281132861306 L 390.0 241.6701374427322 L 394.0 239.92086630570594 L 398.0 238.00499791753435 L 402.0 235.92253227821743 L 406.0 233.67346938775512 L 410.0 231.25780924614745 L 414.0 228.67555185339444 L 418.0 225.92669720949604 L 422.0 223.01124531445231 L 426.0 219.92919616826322 L 430.0 216.68054977092879 L 434.0 213.26530612244898 L 438.0 209.68346522282383 L 442.0 205.93502707205332 L 446.0 202.01999167013744 L 450.0 197.93835901707621 L 454.0 193.69012911286964 L 458.0 189.27530195751771 L 462.0 184.69387755102045 L 466.0 179.94585589337777 L 470.0 175.03123698458978 L 474.0 169.95002082465641 L 478.0 164.7022074135777 L 482.0 159.2877967513536 L 486.0 153.70678883798416 L 490.0 147.9591836734694 L 494.0 142.04498125780927 L 498.0 135.96418159100375 L 502.0 129.7167846730529 L 506.0 123.30279050395671 L 510.0 116.72219908371514 L 514.0 109.97501041232822 L 518.0 103.06122448979593 L 522.0 95.980841316118301 L 526.0 88.733860891295336 L 530.0 81.320283215326981 L 534.0 73.740108288213264 L 538.0 65.993336109954186 L 542.0 58.079966680549774 L 546.0 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 10 20 30 40 50 This label overlaps two sets of axes!

... remember when placing labels directly on the canvas that, unlike Cartesian coordinates, canvas coordinates increase from top-to-bottom.

## Coordinate System Color Scales

Since we often use color in visualization to add an additional dimension to our plots, we need a way to help viewers map between colors and values.  For this case, Toyplot allows a color scale to be added to a set of :ref:`cartesian-coordinates`:

In [6]:
data = toyplot.data.cars()

colormap = toyplot.color.brewer.map(
    name="BlueGreenBrown",
    reverse=True,
    domain_min = data["MPG"].min(),
    domain_max = data["MPG"].max(),
)
canvas = toyplot.Canvas(width=600, height=400)
axes = canvas.cartesian(xlabel="Year", ylabel="Horsepower", margin=75)
axes.scatterplot(
    data["Year"],
    data["Horsepower"],
    color=(data["MPG"], colormap),
    size=8,
    mstyle={"stroke":"black", "stroke-opacity":0.3}
)
axes.color_scale(colormap, label="MPG");

70 74 78 82 Year 50 100 150 200 250 Horsepower 10 20 30 40 50 MPG

Note that a colormap must be explicitly specified when creating a color scale - this is necessary to avoid ambiguity when a single coordinate system contains multiple visualizations or data series.

## Canvas Color Scales

For situations where displaying a vertical color scale with a single set of Cartesian axes is too limiting, you can add horizontal color scales directly to a canvas using any :ref:`canvas-layout` that makes sense.  For example, the following figure uses a single horizontal color scale to display a colormap that is shared between two coordinate systems:

In [7]:
canvas = toyplot.Canvas(width=600, height=400)

axes = canvas.cartesian(bounds=("10%", "45%", "10%", "65%"), xlabel="Year", ylabel="Horsepower")
axes.scatterplot(
    data["Year"],
    data["Horsepower"],
    color=(data["MPG"], colormap),
    size=8,
    mstyle={"stroke":"black", "stroke-opacity":0.3}
)

axes = canvas.cartesian(bounds=("-45%", "-10%", "10%", "65%"), xlabel="Weight", ylabel="Horsepower")
axes.y.spine.position="high"
axes.scatterplot(
    data["Weight"],
    data["Horsepower"],
    color=(data["MPG"], colormap),
    size=8,
    mstyle={"stroke":"black", "stroke-opacity":0.3}
)

canvas.color_scale(colormap, bounds=("10%", "-10%", "75%", "90%"), label="MPG");


70 74 78 82 Year 50 100 150 200 250 Horsepower 2000 3000 4000 5000 Weight 50 100 150 200 250 Horsepower 10 20 30 40 50 MPG

Note that when manually adding a color scale to a canvas, you can orient it any way you like (including diagonally!) by explicitly specifying the endpoints in canvas coordinates:

In [8]:
colormap = toyplot.color.brewer.map(
    name="Spectral",
    domain_min=0,
    domain_max=1,
)
canvas = toyplot.Canvas(width=400)
canvas.color_scale(colormap, x1=50, y1=-50, x2=50, y2=50, label="Bottom to Top")
canvas.color_scale(colormap, x1=150, y1=50, x2=150, y2=-50, label="Top to Bottom")
canvas.color_scale(colormap, x1=200, y1=150, x2=350, y2=150, label="Left to Right")
canvas.color_scale(colormap, x1=350, y1=250, x2=200, y2=250, label="Right to Left");

0.0 0.5 1.0 Bottom to Top 0.0 0.5 1.0 Top to Bottom 0.0 0.5 1.0 Left to Right 0.0 0.5 1.0 Right to Left

## Canvas Legends

Last-but-not-least, Toyplot provides basic support for graphical legends:

In [9]:
observations = numpy.random.power(2, size=(50, 50))

x = numpy.arange(len(observations))

boundaries = numpy.column_stack(
    (numpy.min(observations, axis=1),
     numpy.percentile(observations, 25, axis=1),
     numpy.percentile(observations, 50, axis=1),
     numpy.percentile(observations, 75, axis=1),
     numpy.max(observations, axis=1)))

color = ["blue", "blue", "red", "red"]
opacity = [0.1, 0.2, 0.2, 0.1]

canvas = toyplot.Canvas(800, 400)
axes = canvas.cartesian(grid=(1,5,0,1,0,4))
fill = axes.fill(x, boundaries, color=color, opacity=opacity)
mean = axes.plot(x, numpy.mean(observations, axis=1), color="blue")

canvas.legend([
    ("Mean", mean),
    ("Quartiles", fill),
    ],
    corner=("right", 100, 100, 50),
    );


<polygon points="50.0,310.529681051571 60.799999999999997,338.2127767728843 71.599999999999994,311.53553288533789 82.400000000000006,322.17043210615105 93.200000000000003,299.81448759025807 104.0,327.32054709954076 114.8,309.40794168444785 125.60000000000001,325.92889561071263 136.40000000000001,301.59217273025746 147.19999999999999,337.66827765002898 158.0,299.90161002270065 168.80000000000001,308.77621001463456 179.59999999999999,296.5515581918126 190.40000000000001,315.32993678759516 201.20000000000002,329.77175301438626 212.0,307.00969965429613 222.80000000000001,345.47755682451844 233.60000000000002,318.59512358790397 244.40000000000001,326.85811079352749 255.19999999999999,278.87555671117883 266.0,323.82687058847671 276.80000000000001,330.85065229962754 287.60000000000002,336.00266120160171 298.40000000000003,331.27306243526573 309.19999999999999,295.41680020903141 320.0,295.03772432933994 330.80000000000001,304.86771847435369 341.60000000000002,335.6064233154388 352.40000000000003,326.83055695896473 363.19999999999999,329.94017522326396 374.0,321.61215724481508 384.80000000000001,308.77566069236133 395.60000000000002,300.72983043200446 406.40000000000003,321.00130270928418 417.20000000000005,309.30492377407103 428.0,308.49798899483767 438.80000000000001,348.03081885948359 449.60000000000002,310.55874250447903 460.39999999999998,307.74029875069022 471.19999999999999,295.76300155064314 482.0,284.7102771688584 492.79999999999995,275.3166503109411 503.59999999999997,290.26400117016368 514.39999999999998,328.0829003038271 525.20000000000005,321.65636423299139 536.0,294.2297217685454 546.80000000000007,320.28285540240495 557.60000000000002,305.47084318156624 568.39999999999998,279.59350772653613 579.20000000000005,339.89354855395294 579.20000000000005,177.05982692073246 568.39999999999998,201.82453339017701 557.60000000000002,228.78252611912851 546.80000000000007,176.84007381383924 536.0,199.7288409939153 525.20000000000005,184.88343792030861 514.39999999999998,208.54701216011208 503.59999999999997,202.76496591342072 492.79999999999995,190.16132728156785 482.0,162.55399241167672 471.19999999999999,194.56945380757668 460.39999999999998,229.90878304398768 449.60000000000002,186.84959820046427 438.80000000000001,215.53837396793469 428.0,219.10523540355584 417.20000000000005,187.98427903568549 406.40000000000003,219.97172833448474 395.60000000000002,190.88017296052618 384.80000000000001,202.86048393247731 374.0,151.41944680044713 363.19999999999999,207.89318183264487 352.40000000000003,187.8578033739974 341.60000000000002,184.77875137982295 330.80000000000001,170.54211777961092 320.0,170.67416705214276 309.19999999999999,226.61981088765177 298.40000000000003,217.12879767584374 287.60000000000002,240.30611862472537 276.80000000000001,238.35762112152938 266.0,188.66478780682553 255.19999999999999,197.04064040942421 244.40000000000001,215.05635258266966 233.60000000000002,222.22761081943116 222.80000000000001,210.32526377147644 212.0,217.27390304117682 201.20000000000002,217.20607871621601 190.40000000000001,241.27848064499705 179.59999999999999,205.43639938205806 168.80000000000001,182.49110182672266 158.0,198.47441917051052 147.19999999999999,253.42408685289939 136.40000000000001,205.45636907396658 125.60000000000001,223.52641820044244 114.8,200.03385763417586 104.0,187.00994987042787 93.200000000000003,155.54598387845036 82.400000000000006,211.8253325570285 71.599999999999994,185.58703931051519 60.799999999999997,169.57578085147924 50.0,225.0809880305612" style="fill:rgb(0%,0%,100%);fill-opacity:1.0;opacity:0.1;stroke:none"> <polygon points="50.0,225.0809880305612 60.799999999999997,169.57578085147924 71.599999999999994,185.58703931051519 82.400000000000006,211.8253325570285 93.200000000000003,155.54598387845036 104.0,187.00994987042787 114.8,200.03385763417586 125.60000000000001,223.52641820044244 136.40000000000001,205.45636907396658 147.19999999999999,253.42408685289939 158.0,198.47441917051052 168.80000000000001,182.491101826

The call to :func:`toyplot.canvas.Canvas.legend` always includes an explicit list of entries to add to the legend, plus a :ref:`canvas-layout` specification of where the layout should appear on the canvas.  Currently, each entry to be displayed in a legend must be one of the following:

* A (label, mark) tuple, which will get its appearance from the mark, or:
* A (label, marker) tuple, which can be used to specify an arbitrary :ref:`Marker<Markers>`.

Of course, `label` is the human-readable text to be displayed next to an item in the legend, while `mark` is a mark that has been added to the canvas.  However, not all marks can map cleanly to a single entry in the legend - note in the example above that the fill mark added multiple markers to the "Quartiles" entry in the legend, one for each data series.  While the interpretation is reasonably clear in this case, there will be occasions when there isn't a sensible one-to-one mapping between a mark and an entry in the legend.  For example, the meaning of multiple series may not be clear, or you may be plotting categorical information using custom markers in a line or scatter plot.  In these cases use the second form of legend entry to specify as many explicit :ref:`Markers` as needed.

There are some subtleties here worth noting, many of which are driven by Toyplot's deliberate embrace of the philosophy that *explicit is better than implicit*:

* You can have as many or as few legends on your canvas as you like.
* Callers explicitly specify the order and contents of each legend.
* There is no relationship between axes and legends - you can combine marks from multiple axes in a single legend.

Here's an example with all these ideas at work.  Note that the legend overlaps two coordinate systems and its first entry derives directly from the mark in the first coordinate system, while the second and third entries document the individual series in the second mark: 

In [10]:
x = numpy.linspace(0, 1)
y1 = (1 - x) ** 2
y2 = numpy.column_stack((1 - (x ** 2), x ** 2))

canvas = toyplot.Canvas(width=600, height=300)
m1 = canvas.cartesian(grid=(1,2,0), margin=25).scatterplot(x, y1, marker="o", color="rgb(255,0,0)")
m2 = canvas.cartesian(grid=(1,2,1), margin=25, yshow=False).scatterplot(x, y2, marker="s", color=["green", "blue"])

if True:
    canvas.legend([
        ("Experiment 1", m1),
        ("Experiment 2", m2.markers[0]),
        ("Experiment 3", m2.markers[1]),
        ],
        corner=("top", 100, 100, 70),
        );

0.0 0.5 1.0 0.0 0.5 1.0 0.0 0.5 1.0 Experiment 1 Experiment 2 Experiment 3